9. Which player has won the most tournaments in a single month? 


In [2]:
import pandas as pd
from pathlib import Path

project_root = Path.cwd().parent
processed_dir = project_root / "data" / "processed"
raw_dir = project_root / "data" / "raw"
out_dir = project_root / "reports" / "answers"
out_dir.mkdir(parents=True, exist_ok=True)

processed_dir.mkdir(parents=True, exist_ok=True)

players_file_processed = processed_dir / "players.parquet"
players_file_raw = raw_dir / "players.parquet"

if not players_file_processed.exists() and players_file_raw.exists():
    df_players_raw = pd.read_parquet(players_file_raw)
    df_players_raw.to_parquet(players_file_processed, index=False)
    print(f"Copied players.parquet from raw to processed: {players_file_processed}")

df_feb = pd.read_parquet(processed_dir / "match_round_info_february.parquet")
df_mar = pd.read_parquet(processed_dir / "match_round_info_march.parquet")
df_results = pd.read_parquet(processed_dir / "match_results_player_filled.parquet")
df_players = pd.read_parquet(players_file_processed)

df_feb["month"] = "February"
df_mar["month"] = "March"

df_rounds = pd.concat([df_feb, df_mar], ignore_index=True)
df_finals = df_rounds[df_rounds["cup_round_type"] == 1.0]
df_merged = df_finals.merge(df_results, on="match_id", how="inner")

def get_winner(row):
    if row["winner_code"] == 1:
        return row["home_player_id"]
    elif row["winner_code"] == 2:
        return row["away_player_id"]
    return None

df_merged["winner_player_id"] = df_merged.apply(get_winner, axis=1)

winners_count = (
    df_merged[["month", "winner_player_id", "match_id"]]
    .drop_duplicates()
    .groupby(["month", "winner_player_id"])
    .size()
    .reset_index(name="titles_won")
)

idx = winners_count.groupby("month")["titles_won"].idxmax()
top_champions_per_month = winners_count.loc[idx].reset_index(drop=True)

top_champions_per_month = top_champions_per_month.merge(
    df_players[["player_id", "slug"]],
    left_on="winner_player_id",
    right_on="player_id",
    how="left"
)

output_path = out_dir / "q9_top_champions_per_month.csv"
top_champions_per_month[["month", "winner_player_id", "slug", "titles_won"]]\
    .to_csv(output_path, index=False, encoding="utf-8-sig")

print(top_champions_per_month[["month", "winner_player_id", "slug", "titles_won"]])
print(f"Saved to {output_path}")


      month  winner_player_id          slug  titles_won
0  February           50901.0  popko-dmitry           3
1     March          152766.0  helgo-malene           3
Saved to c:\Users\mit\Desktop\Finaaal\reports\answers\q9_top_champions_per_month.csv


In [3]:
overall_winner = (
    winners_count.groupby("winner_player_id", as_index=False)["titles_won"]
    .sum()
    .sort_values(by="titles_won", ascending=False)
    .head(1)
    .merge(
        df_players[["player_id", "slug"]],
        left_on="winner_player_id",
        right_on="player_id",
        how="left"
    )
)

output_overall_path = out_dir / "q9_overall_top_champion.csv"
overall_winner[["winner_player_id", "slug", "titles_won"]].to_csv(
    output_overall_path, index=False, encoding="utf-8-sig"
)

print("\nTop champion overall (Feb + Mar):")
print(overall_winner[["winner_player_id", "slug", "titles_won"]])
print(f"Saved to {output_overall_path}")



Top champion overall (Feb + Mar):
   winner_player_id          slug  titles_won
0           50901.0  popko-dmitry           4
Saved to c:\Users\mit\Desktop\Finaaal\reports\answers\q9_overall_top_champion.csv
